# Interacting with Coq

The `Pytanque` class implements a lightweight client that can communicate with the Rocq prover via the `pet-server`.

In [10]:
from pytanque import Pytanque

First you need to start the server.
Instructions can be found in the coq-lsp repo (https://github.com/ejgallego/coq-lsp/tree/main/petanque)

Then, you can create a client and connect it to the same address and port.
You need to indicate the location of the project (directory containing the `_CoqProject` file, or root directory for the `.v` files).

In [11]:
pet = Pytanque("127.0.0.1", 8765)
pet.connect()
env = pet.init(root=".")

Petanque relies on regular coq files.
To prove an existing theorem, just indicate the location of the `.v` file and the name of the theorem.

In [12]:
state = pet.start(file="./foo.v", thm="addnC")

Internally the state is a simple `id`.
To retrieve the state of the prover, you can use the `goals` method.

In [13]:
print(f"Internal pytanque state: {state}")

goals = pet.goals(state)  # Retrieve actual state
print(f"Prover goals: {goals}\n")
goals.pp()  # Pretty print the current goals

Internal pytanque state: CurrentState(value=7)
Prover goals: GoalsResponse(goals=[Goal(info={'evar': ['Ser_Evar', 12], 'name': None}, hyps=[GoalHyp(names=['n', 'm'], ty='nat', def_=None)], ty='n + m = m + n')])

n, m  : nat
-----------------------------
n + m = m + n





To execute a tactic, use the `run_tac` method.

In [14]:
state = pet.run_tac(state, "induction n.")
goals = pet.goals(state)
goals.pp()

m  : nat
-----------------------------
0 + m = m + 0



n, m  : nat
IHn  : n + m = m + n
-----------------------------
S n + m = m + S n





If the proof is finished (no more goal) the state becomes `ProofFinished`

In [15]:
state = pet.start(file="./foo.v", thm="addnC")
state = pet.run_tac(state, "by elim: n => //= ? ->.")
state

ProofFinished(value=10)

# Proof search

Adapted from the NeurIPS 2024 tuorial: https://github.com/yangky11/ml4tp-tutorial/blob/main/main.ipynb 

Let us start with a dummy tactic generator (random sampling in a list of tactics).

In [16]:
import random


def generate_tactics(goal: str, num_candidates: int):
    tactics = [
        "intros.",
        "auto.",
        "simpl.",
        "induction n.",
        "easy.",
        "trivial.",
        "rewrite IHn.",
    ]
    return random.choices(tactics, k=num_candidates)

We can now implement depth-first search to search for proofs.

In [17]:
from typing import List, Optional
import pytanque as pt

Tactic = str
Proof = List[Tactic]

num_candidates = 16
depth_limit = 3


def dfs_search(state, depth: int) -> Optional[Proof]:
    """Try to prove `state` using depth-first search (DFS)."""
    if depth >= depth_limit:
        return None

    tactics = generate_tactics(state, num_candidates)

    # Run the tactics.
    for tac in tactics:
        try:
            next_state = pet.run_tac(state, tac)
            match next_state:
                case pt.ProofFinished(st):
                    return [tac]
                case pt.CurrentState(st):
                    subproof = dfs_search(next_state, depth + 1)
                    if subproof:
                        return [tac] + subproof
        except pt.PetanqueError:
            pass
    return []

Let us try on a simple example.

In [18]:
state = pet.start(file="./foo.v", thm="t")
proof = dfs_search(state, depth=0)

if proof:
    print(f"Proof: {" ".join(proof)}")
else:
    print("Not proof found...")

Proof: intros. intros. auto.
